In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)


import numpy as np
from random import randint
from math import floor, ceil
import codecs
from epiweeks import Week
from glob import glob

In [ ]:
from bulletin import default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label, tables_path
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace

from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)

In [ ]:
tables = dict([(Path(x).stem,pd.read_csv(join(tables_path,x))) for x in glob(join(tables_path,"*.csv"))])

In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{today.strftime('%d_%m_%Y')}", compress=True)
cc.df

In [ ]:
cc.df = cc.df.join(tables['municipios'].set_index('ibge'), on='ibge_residencia')

In [ ]:
casos_confirmados = cc.df.loc[cc.df['uf']=='PR'].copy()

In [ ]:
casos_confirmados = casos_confirmados.rename(columns={'id':'id'})
casos_confirmados['ano_caso'] = casos_confirmados['data_diagnostico'].apply(lambda x: x.year if isinstance(x,date) else 'erro')
casos_confirmados['mes_caso'] = casos_confirmados['data_diagnostico'].apply(lambda x: x.month if isinstance(x,date) else 'erro')
casos_confirmados['se_caso'] = casos_confirmados['data_diagnostico'].apply(lambda x: Week.fromdate(x).week if isinstance(x,date) else 'erro')
casos_confirmados['dia_caso'] = casos_confirmados['data_diagnostico'].apply(lambda x: x.day if isinstance(x,date) else 'erro')

obitos_confirmados = casos_confirmados.loc[casos_confirmados['evolucao']==2].copy()
obitos_confirmados['ano_obito'] = obitos_confirmados['data_cura_obito'].apply(lambda x: x.year if isinstance(x,date) else 'erro')
obitos_confirmados['mes_obito'] = obitos_confirmados['data_cura_obito'].apply(lambda x: x.month if isinstance(x,date) else 'erro')
obitos_confirmados['se_obito'] = obitos_confirmados['data_cura_obito'].apply(lambda x: Week.fromdate(x).week if isinstance(x,date) else 'erro')
obitos_confirmados['dia_obito'] = obitos_confirmados['data_cura_obito'].apply(lambda x: x.day if isinstance(x,date) else 'erro')

In [ ]:
datas = pd.DataFrame(index=pd.date_range(casos_confirmados['data_diagnostico'].min(),today))
datas.index.names = ['data']
datas.loc[:,['ano','mes','se','dia']] = [ (data.year, data.month, Week.fromdate(data).week, data.day) for data in datas.index ]
datas = datas.set_index(['ano','mes','se','dia'])
datas

In [ ]:
casos = casos_confirmados.groupby(['macro','rs','regional','ibge_residencia','municipio','ano_caso','mes_caso','se_caso','dia_caso'])[['id']].count().fillna(0).astype(int).rename(columns={'id':'casos'}).unstack(['ano_caso','mes_caso','se_caso','dia_caso']).droplevel(0,1)
casos.columns.names = ['ano','mes','se','dia']
casos = casos.reindex(datas.index, axis=1).sort_index(level=1).fillna(0).astype(int)
casos

In [ ]:
obitos = obitos_confirmados.groupby(['macro','rs','regional','ibge_residencia','municipio','ano_obito','mes_obito','se_obito','dia_obito'])[['id']].count().fillna(0).astype(int).rename(columns={'id':'obito'}).unstack(['ano_obito','mes_obito','se_obito','dia_obito']).sort_index().droplevel(0,1)
obitos.columns.names = ['ano','mes','se','dia']
obitos = obitos.reindex(datas.index, axis=1).sort_index(level=1).fillna(0).astype(int)
obitos

In [ ]:
casos.columns = pd.MultiIndex.from_product([['casos'],casos.columns])
obitos.columns = pd.MultiIndex.from_product([['obitos'],obitos.columns])

casos_obitos = casos.join(obitos).reorder_levels(['ano','mes','se','dia','qtde'], axis=1).sort_index(level=0, axis=1)
casos_obitos